In [1]:
import sys
import os
sys.path.append("engine/")
sys.path.append("funcs/")
sys.setrecursionlimit(10000)

import theano.gpuarray
theano.gpuarray.use('cuda0')

import theano
import theano.tensor as T
from theano import config
import numpy as np
import pickle
import copy
import time
from itertools import *

from matplotlib import pyplot as plt
from matplotlib import gridspec
%matplotlib inline

from VIMCO_alpha import *

import xlrd
import pandas as pd
import time
import openpyxl

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/root/miniconda3/envs/spike/lib/python3.6/site-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7604 on context None
Mapped name None to device cuda0: Tesla V100-PCIE-32GB (0000:65:02.0)


In [2]:
config.gpu_preallcoate = 0.9

In [ ]:
import torch
model_500 = torch.load('./model_500.pt')

In [ ]:
def plot_preds_bl_cr_v2(model, data, cell=0, pred=True, ts=[0, 1000], trace=0, figsize=None, GS=None, prob=0.25):
    # print('cr')
    n_samples = 13

    # if figsize is None:
    #     fig = plt.figure()
    # elif isinstance(figsize, tuple):
    #     fig = plt.figure(figsize=figsize)
    # else:
    #     fig = figsize

    cp = sns.hls_palette(11, l=.4, s=.8)
    try:
        sr = model.superres
    except KeyError:
        sr = 1

    start = 0
    # end = - 1
    end = len(data['spikes'][cell][trace])

    fluor, spikes = data_resamp(data['traces'], data['spikes'], data['fps'], data['spike_fps'], model.resample)
    fluor = np.array(fluor[cell][trace][ts[0]:ts[1]], ndmin=2).astype(config.floatX)
    spikes = np.array(spikes[cell][trace][sr * ts[0]:sr * ts[1]], ndmin=2).astype(config.floatX)

    # model.mgen.load_genparams(cell)
    model.mgen.load_genparams(0)
    rec_dict = model.mrec.get_sample(fluor, n_samples)
    pred_prob = rec_dict['Probs']
    # print(len(pred_prob[0]))
    pred_spikes = rec_dict['Spikes']
    pred_bl = 0

    
    dt = 1 / model.resample / sr
    c = 1
    t = np.arange(len(fluor[0, start:end])) * dt
    t_sr = np.arange(len(fluor[0, start:end]) * sr) * dt / sr
    
    if model.mrec.n_genparams:
        pred_pars = rec_dict['Params']
    
    # print(sr * start, sr * end, sr)
    # pred_prob = pred_prob[:, sr * start:sr * end:sr].mean(axis=0)
    pred_prob = pred_prob.mean(axis=0)
    
    
    # pred_prob = np.where(pred_prob > prob, pred_prob, 0)
    pred_prob = np.where(pred_prob > prob, 1, 0)
    
    return t_sr, pred_prob
   

In [7]:
def main(data_path, folder):
    path = []
    for i in os.listdir(data_path):    
        if i[-5:] == '.xlsx':
            path.append(data_path+'/'+i)
    # print(path)
    n_cells = len(path)
    spikes_raw = [[] for x in range(n_cells)]
    traces_raw = [[] for x in range(n_cells)]
    fps = [[] for x in range(n_cells)]

    for k in range(len(path)): 
        # print(k)
        wb = xlrd.open_workbook(path[k])
        # sheet = wb.sheet_by_name('Sheet1')
        sheet = wb.sheet_by_index(0)
        # if k < 9:
        #     sheet = wb.sheet_by_name('Results00{}'.format(path(k)[-6]+1))
        # else:
        #     sheet = wb.sheet_by_name('Results0{}'.format(k+1))
        one_data =np.zeros([sheet.nrows-1,sheet.ncols])
        print(sheet.nrows-1,sheet.ncols)
        for i in range(1,sheet.nrows):
            for j in range(sheet.ncols):
                one_data[i-1,j]=sheet.cell(i,j).value
        # calcium_traces = []
        for c in range(one_data.shape[1]):
            calcium_trace = one_data[:, c]

            spikes_raw[k].append(calcium_trace*0)
            traces_raw[k].append(calcium_trace)
            fps[k].append(50)
    for i in range(len(fps)):
        fps[i] = np.mean(fps[i])

    spikefps = 50
    data_dict = {'traces': traces_raw, 'spikes': spikes_raw, 'fps' : fps, 'spike_fps': spikefps}


    for i in range(len(data_dict['traces'])):
        for j in range(len(data_dict['traces'][i])):
            # for k in range(len(data_dict['traces'][i][j])):
            # print(len(data_dict['traces'][i][j]))
            min_value = np.min(data_dict['traces'][i][j])
            max_value = np.max(data_dict['traces'][i][j])
            data_dict['traces'][i][j] = (data_dict['traces'][i][j] - min_value) / (max_value - min_value)
            
    print('numbers of cell:', len(data_dict['traces']))
    for c in range(len(data_dict['traces'])):
    # for c in range(1):
        print('Cell ', c)
        t1 = time.time()
        output = []
        for t in range(len(data_dict['traces'][c])):
            # plot_preds_bl_cr_v2(model_500,data_smooth,c,trace=t,figsize=(35,7),ts=[0,1400],prob=0)
            t_sr, pred_prob = plot_preds_bl_cr_v2(model_500,data_dict,c,trace=t,figsize=(35,7),ts=[0,10000],prob=0.1)
            output.append(pred_prob)
            # plt.title('cell:{0} trace: {1}'.format(c, t))
            # plt.savefig('/root/autodl-tmp/DeepSpike-master/output2' + folder + '/cell:{0} trace: {1}.png'.format(c, t))
            # plt.show()
        output = pd.DataFrame(output)
        # output.T.to_csv("./data0509_prepped_2_smooth.csv" ,index=False)
        output.T.to_excel('/root/autodl-tmp/DeepSpike-master/OUTPUT/output_part6/' + folder + '/' + path[c][67:], index = False, header=None)
        print(time.time()-t1)
        
    return "over"

In [8]:
main('/root/autodl-tmp/DeepSpike-master/dataset/data_part_6/intensity_24', 'intensity_24')

1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
1800 80
numbers of cell: 62
Cell  0
80.90397429466248
Cell  1
85.07608842849731
Cell  2
85.82276391983032
Cell  3
85.80850148200989
Cell  4
85.94899153709412
Cell  5
81.68708038330078
Cell  6
80.67267227172852
Cell  7
77.92069292068481
Cell  8
77.72527480125427
Cell  9
74.58281922340393
Cell  10
75.76309990882874
Cell  11
78.00790143013
Cell  12
76.99556636810303
Cell  13
76.45406937599182
Cell  14
80.20191216468811
Cell  15
76.40134286880493
Cell  16
76.89834642410278
Cell  17
76.98259735107422
Cell  18
81

'over'

In [ ]:
root = '/root/autodl-tmp/DeepSpike-master/dataset/data_part_6'
t3 = time.time()
for i in os.listdir(root):
    print("文件夹：", i)
    t2 = time.time()
    data_path = root + '/' + i
    print(data_path)
    main(data_path, i)
    print("时间：", time.time() - t2)
print('总时间：', time.time() - t3)

文件夹： intensity_23
/root/autodl-tmp/DeepSpike-master/dataset/data_part_6/intensity_23
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1800 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
1801 179
numbers of cell: 61
Cell  0
835.1832871437073
Cell  1
832.5707352161407
Cell  2


In [8]:
path = []
for i in os.listdir(data_path): 
    print(i)
    if i[-5:] == '.xlsx':
        path.append(data_path+'/'+i)
print(path)

intensity_1.xlsx
intensity_2.xlsx
intensity_3.xlsx
intensity_4.xlsx
intensity_5.xlsx
intensity_6.xlsx
['/root/autodl-tmp/DeepSpike-master/dataset/data_part_5/intensity_31/intensity_1.xlsx', '/root/autodl-tmp/DeepSpike-master/dataset/data_part_5/intensity_31/intensity_2.xlsx', '/root/autodl-tmp/DeepSpike-master/dataset/data_part_5/intensity_31/intensity_3.xlsx', '/root/autodl-tmp/DeepSpike-master/dataset/data_part_5/intensity_31/intensity_4.xlsx', '/root/autodl-tmp/DeepSpike-master/dataset/data_part_5/intensity_31/intensity_5.xlsx', '/root/autodl-tmp/DeepSpike-master/dataset/data_part_5/intensity_31/intensity_6.xlsx']


In [32]:
'/root/autodl-tmp/DeepSpike-master/dataset/data_part_6/intensity_23/intensity_1_mean.xlsx'
wb = xlrd.open_workbook('/root/autodl-tmp/DeepSpike-master/dataset/data_part_6/intensity_23/intensity_1_mean.xlsx')
# sheet = wb.sheet_by_name('Sheet1')
sheet = wb.sheet_by_index(0)
# if k < 9:
#     sheet = wb.sheet_by_name('Results00{}'.format(path(k)[-6]+1))
# else:
#     sheet = wb.sheet_by_name('Results0{}'.format(k+1))
one_data =np.zeros([sheet.nrows,sheet.ncols])
for i in range(1, sheet.nrows):
    for j in range(1, sheet.ncols):
        one_data[i,j]=sheet.cell(i,j).value
# calcium_traces = []